In [11]:
import os
import sys
import time
import serial
import math
import numpy as np

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
sys.path.insert(0, os.path.join(os.getcwd(), 'utils'))
from Thymio import Thymio

In [12]:
class robot():
    def __init__(self, th):
        self.curr_pos = [0,0,0]
        self.speed = 100
        self.th = th
        
    def get_position(self):
        return self.curr_pos
        
    def move_to_target(self, target_pos):
        if target_pos == self.curr_pos[0:2]: return False #if the robot is already at the position or doesn't move
       
        #distance from current position to target
        distance = math.sqrt(((target_pos[0]-self.curr_pos[0])**2)+((target_pos[1]-self.curr_pos[1])**2))
        
        #absolute angle from current position to target (this angle will always be returned between ]-pi;pi])
        path_angle = math.atan2(target_pos[1]-self.curr_pos[1],target_pos[0]-self.curr_pos[0])
        
        #turn angle to get to target relative to Thymio frame
        turn_angle = path_angle - self.curr_pos[2]
        
        print("distance:{}".format(distance))
        print("turn_angle:{}".format(math.degrees(turn_angle)),"\n")
        
        #give commands
        self.turn(turn_angle)
        self.forward(distance)

        #update position and angle of the robot
        self.curr_pos = [target_pos[0],target_pos[1],path_angle]
        
    def turn(self, turn_angle):
        if turn_angle > 0: #turn_angle to the left (180 case will also turn left)
            target_time = (abs(math.degrees(turn_angle)))/38.558 #linear fit model from rad to s for v=100 (change to Kalman)
            left_speed = 2**16-self.speed 
            right_speed = self.speed
            
        elif turn_angle < 0: #turn_angle to the right
            target_time = (abs(math.degrees(turn_angle)))/38.558 
            left_speed = self.speed
            right_speed = 2**16-self.speed
            
        else: #if turn_angle = 0, do not waste time
            return False
            
        print("target_time_turn:{}".format(target_time))
        t_0 = time.time()
        
        self.th.set_var("motor.left.target", left_speed)
        self.th.set_var("motor.right.target", right_speed)
        
        time_move(target_time)
        t_end = time.time()
        print("actual_time_turn:{}".format(t_end-t_0))
        
        #time.sleep(0.1)

    def forward(self, distance): 
        target_time = distance/31.573 #linear fit model from mm to s for v=100 (change to Kalman)
        
        print("target_time_forward:{}".format(target_time))
        t_0 = time.time()
        
        self.th.set_var("motor.left.target", self.speed)
        self.th.set_var("motor.right.target", self.speed)
        
        time_move(target_time)
        t_end = time.time()
        print("actual_time_forward:{}".format(t_end-t_0), "\n")
        
        #time.sleep(0.1)
        
    def stop(self):
        self.th.set_var("motor.left.target", 0)
        self.th.set_var("motor.right.target", 0)
        
        #time.sleep(0.1)
    def time_move(target_time):
        t=0
        while t < target_time:
            time.sleep(0.1)
            t += 0.1
            #local_nav
            if test_saw_wall():
                a=1

    def test_saw_wall(wall_threshold=500, verbose=False))
        return False

In [14]:
th = Thymio.serial(port="/dev/ttyACM3", refreshing_rate=0.1) #/dev/ttyACM0 for linux #/dev/cu.usbmodem141401 for mac

th_m = robot(th)
time.sleep(3) # To make sure the Thymio has had time to connect

variables = th.variable_description()
for var in variables : #If this does not print, reconnect USB!
    print(var)


{'name': '_id', 'offset': 0, 'size': 1}
{'name': 'event.source', 'offset': 1, 'size': 1}
{'name': 'event.args', 'offset': 2, 'size': 32}
{'name': '_fwversion', 'offset': 34, 'size': 2}
{'name': '_productId', 'offset': 36, 'size': 1}
{'name': 'buttons._raw', 'offset': 37, 'size': 5}
{'name': 'button.backward', 'offset': 42, 'size': 1}
{'name': 'button.left', 'offset': 43, 'size': 1}
{'name': 'button.center', 'offset': 44, 'size': 1}
{'name': 'button.forward', 'offset': 45, 'size': 1}
{'name': 'button.right', 'offset': 46, 'size': 1}
{'name': 'buttons._mean', 'offset': 47, 'size': 5}
{'name': 'buttons._noise', 'offset': 52, 'size': 5}
{'name': 'prox.horizontal', 'offset': 57, 'size': 7}
{'name': 'prox.comm.rx._payloads', 'offset': 64, 'size': 7}
{'name': 'prox.comm.rx._intensities', 'offset': 71, 'size': 7}
{'name': 'prox.comm.rx', 'offset': 78, 'size': 1}
{'name': 'prox.comm.tx', 'offset': 79, 'size': 1}
{'name': 'prox.ground.ambiant', 'offset': 80, 'size': 2}
{'name': 'prox.ground.refl

In [15]:
my_th = robot(th)

pos_th = my_th.get_position()
print("Thymio position: {}".format(pos_th))
my_th.move_to_target([0,100])

pos_th = my_th.get_position()
print("Thymio position: {}".format(pos_th))
my_th.move_to_target([50,150])

pos_th = my_th.get_position()
print("Thymio position: {}".format(pos_th))
my_th.move_to_target([50,300])

pos_th = my_th.get_position()
print("Thymio position: {}".format(pos_th))
my_th.move_to_target([70,400])

pos_th = my_th.get_position()
print("Thymio position: {}".format(pos_th))
    
my_th.stop()
        

Thymio position: [0, 0, 0]
distance:100.0
turn_angle:90.0 

target_time_turn:2.334145961927486
actual_time_turn:2.337470769882202
target_time_forward:3.1672631678966203
actual_time_forward:3.1689999103546143 

Thymio position: [0, 100, 1.5707963267948966]
distance:70.71067811865476
turn_angle:-45.0 

target_time_turn:1.167072980963743
actual_time_turn:1.1689727306365967
target_time_forward:2.239593263822087
actual_time_forward:2.241382122039795 

Thymio position: [50, 150, 0.7853981633974483]
distance:150.0
turn_angle:45.0 

target_time_turn:1.167072980963743
actual_time_turn:1.1708009243011475
target_time_forward:4.750894751844931
actual_time_forward:4.75584602355957 

Thymio position: [50, 300, 1.5707963267948966]
distance:101.9803902718557
turn_angle:-11.309932474020208 

target_time_turn:0.2933225912656312
actual_time_turn:0.2947273254394531
target_time_forward:3.229987339557714
actual_time_forward:3.2311768531799316 

Thymio position: [70, 400, 1.373400766945016]


In [ ]:
#     def measurement_model(self):
#         #distance
#         x_d = np.linspace(0.5, 5, 10)
#         y_d = [16,32.5,46.5,61,79.5,95,109,127.5,143,159.5]
        
#         diff_y_d = np.diff(y_d) # Calculating difference list 
#         mean_d = np.mean(diff_y_d) #15.94444
#         sigma_d = np.std(diff_y_d) #6.877318
#         distance_p0 = [1, mean_d, sigma_d]

#         #angle
#         x_a = np.linspace(0, 9, 10)
#         y_a = [0,42,78,114,150,194,231,270,310,347]

#         diff_y_a = np.diff(y_a) # Calculating difference list 
#         mean_a = np.mean(diff_y_a) 
#         sigma_a = np.std(diff_y_a) 
#         angle_p0 = [1, mean_a, sigma_a]
#         return distance_p0, angle_p0
    
    
    
    
# distance_p0, angle_p0 = th_m.measurement_model()
# print("distance measurement model: {}\n".format(distance_p0))
# print("angle measurement model: {}".format(angle_p0))

